# Fidelity and entanglement

## State fidelity

The fidelity is a measure of how close two states are to each other. The general definition for density matrices is
$${\cal F}(\rho_1,\rho_2) = \left[\mathrm{tr}\sqrt{\sqrt{\rho_1}\rho_2\sqrt{\rho_1}}\right]^2$$

If both states are pure, $\rho_i = |\psi_i\rangle\langle\psi_i|$ we can use $\rho^{1/2}_i=\rho_i$ to obtain
$${\cal F}(\psi_1,\psi_2) = \left|\langle\psi_1|\psi_2\rangle\right|^2.$$

In [ ]:
# file: seeq/fidelity.py
import numpy as np
import scipy.linalg

def state_fidelity(σ1, σ2, normalize=False):
    """Compute the fidelity between states σ1 and σ1, which may be either
    vectors (pure states) or density matrices. Neither the states nor the
    density matrices need be normalized."""
    if σ1.ndim == 2:
        if normalize:
            σ1 /= np.trace(σ1)
        if σ2.ndim == 1:
            #
            # σ1 is a matrix, σ2 is a pure state
            if normalize:
                σ2 = σ2 / np.linalg.norm(σ2)
            return abs(np.vdot(σ2, σ1 @ σ2))
        elif σ2.ndim == 2:
            if normalize:
                σ2 /= np.trace(σ2)
            #
            # Both σ1 and σ2 are density matrices
            #
            λ1, U1 = scipy.linalg.eigh(σ1, overwrite_a=True)
            sqrtσ1 = (U1 * np.sqrt(np.abs(λ1))) @ U1.T.conj()
            λ, U = scipy.linalg.eigh(sqrtσ1 @ σ2 @ sqrtσ1, overwrite_a=True)
            return np.sum(np.sqrt(np.abs(λ)))**2
    elif σ2.ndim == 1:
        #
        # Both are pure states
        F = abs(np.vdot(σ1, σ2))**2
        if normalize:
            return F / (np.linalg.norm(σ1)*np.linalg.norm(σ2))
        else:
            return F
    elif σ2.ndim == 2:
        #
        # σ1 is a pure state, σ2 a density 
        if normalize:
            σ2 /= np.trace(σ2)
            σ1 = σ1 / np.linalg.norm(σ1)
        return abs(np.vdot(σ1, σ2 @ σ1))
    raise ValueException(f'state_fidelity() got neither a pure state nor a density matrix')

## Average fidelity

Following [M. A. Nielsen, Phys. Lett. A 303(4), 249-252 (2002)](https://doi.org/10.1016/S0375-9601(02)01272-0), if we have a positive map $\mathcal{E}(\rho),$ we quantify the average fidelity of this map as
$$\bar{F}[\mathcal{E}] = \int \langle\psi|\mathcal{E}(|\psi\rangle\langle\psi|)|\psi\rangle \mathrm{d}\psi,$$
where the integration happens over the uniform Haar measure over pure states $|\psi\rangle.$

Similarly, there is the entanglement fidelity, defined as
$$F_e[\mathcal{E}] = \langle\phi|(\mathbb{I}\otimes \mathcal{E})(|\phi\rangle\langle\phi|)|\phi\rangle,$$
where $\phi$ represents a maximally entangled state, such as
$$|\phi\rangle = \sum_{i=1}^d \frac{1}{\sqrt{d}}|i,i\rangle,$$
established over a duplicate $\mathcal{H}\otimes\mathcal{H}$ of the Hilbert space on which $\mathcal{E}$ is defined, with $d=\mathrm{dim}\mathcal{H}.$

Both definitions of fidelity are related by Nielsen's formula
$$\bar{F} = \frac{d F_e + 1}{d+1},$$
implying that both fidelities coincide in the limit of infinitely large Hilbert spaces.

### a) Comparing unitaries

If we care about unitaries and want to compare $U$ with the ideal $W,$ our map will be
$$\mathcal{E}(\rho) = U^\dagger W \rho (U^\dagger W)^\dagger = E\rho E^\dagger.$$
with the product $E=U^\dagger W.$

The entanglement fidelity becomes
\begin{eqnarray*}
F_e[{\cal E}] &=& \frac{1}{d^2}\sum_{ijkl}  \langle{i,i}| \left[ |j\rangle\langle{k}|\otimes (E |j\rangle\langle{k}|E^\dagger) \right] |l,l\rangle \\
&=& \frac{1}{d^2}\sum_{jk} E_{jj} E_{kk}^* = \frac{1}{d^2}\left|\mathrm{tr}(E)\right|^2.
\end{eqnarray*}

The entanglement fidelity becomes
$$F_e[U,W]= \frac{1}{d^2}\left|\mathrm{tr}(U W^\dagger)\right|^2.$$
And according to Nielsen's work, the average fidelity becomes
$$\bar{F}[U,W] = \frac{d F_e[U,W]+1}{d+1}.$$

The formula for the average gate fidelity

In [ ]:
# file: seeq/fidelity.py

def avg_unitary_fidelity(U, W=None):
    """How close U is to W (which defaults to identiy)"""
    if W is not None:
        U = U * W.T.conj()
    d = len(U)
    Fe = np.abs(np.trace(U)/d)**2
    F = (d*Fe+1)/(d+1)
    return F

### b) Comparing positive maps

A more general case is one in which ${\cal E}$ converts the states into density matrices, introducing decoherence. In that case, we can use a representation of ${\cal E}$ as a linear superoperator from density matrices to density matrices.
$${\cal E}(\rho)_{ij} = \sum_{kl} {\cal E}_{ij,kl} \rho_{kl}$$

The entanglement fidelity above now becomes
\begin{eqnarray*}
F_e[{\cal E}] &=& \frac{1}{d^2}\sum_{ijkl} \langle{i,i}| \left[|j\rangle\langle{k}|\otimes {\cal E}(|j\rangle\langle{k}|)\right] |l,l\rangle \\
&=& \frac{1}{d^2}\sum_{i,l} \langle{i}|{\cal E}(|i\rangle\langle{l}|) |l\rangle \\
&=& \frac{1}{d^2}\sum_{i,l} {\cal E}_{il,il}.
\end{eqnarray*}

Note that for a unitary transformation, ${\cal E}_{ij,kl}= U_{ik} U_{jl}^*$ and the formula reduces to the one above.

In [ ]:
# file: seeq/fidelity.py

def avg_superoperator_fidelity(E):
    """Return the average fidelity of superoperator E, represented as a four
    dimensional tensor with indices of size d, where 'd' is the size of the
    Hilbert space."""
    if E.ndim == 4:
        d = E.shape[0]
        E = E.reshape(d*d,d*d)
    else:
        raise ValueException('Not a valid representation for a superoperator.')
    Fe = abs(np.trace(E))/(d*d)
    F = (d*Fe+1)/(d+1)
    return F

### c) Combined

In [ ]:
# file: seeq/fidelity.py

def avg_fidelity(T):
    """Return the average fidelity of a transformation T.
    
    Arguments
    ---------
    T  -- Either a 4 dimensional tensor, representing a positive map, or
          a 2 dimensional tensor or matrix, representing a unitary operation.

    Output
    ------
    F  -- A value in [0,1] representing the average fidelity.
    """
    if T.ndim == 2:
        return avg_unitary_fidelity(T)
    elif T.ndim == 4:
        return avg_superoperator_fidelity(T)
    else:
        raise ValueException('Not a valid superoperator or matrix.')

## Leakage

Qubits are usually implemented as a select subspace of a higher-dimensional object, such as a transmon qubit. Our set of states where we store information is called the computational subspace. When we study how to implement gates in such objects, we often find that there is a little probability that our information escapes the computational subspace.

In those cases, we usually compute a /scattering matrix/, which is the projection of the unitary evolution operator onto the computational subspace. Let us assume we have a subspace $V=\mathrm{lin}\{\phi_i\}$ defined with a basis of states $\phi_i.$ We have computed the scattering matrix
$$S_{ij} = \langle \phi_i| U(t)|\phi_j\rangle.$$
In the ideal case, $S$ would become our desired quantum gate and $S_{ij} = U^{\mathrm{ideal}}_{ij}.$ In many cases, however, there will be transitions to states outside $V$.

We wish to quantify the average probability that a state $\xi\in V$ escapes the Hilbert space. We define the leakage as the average of that probability for all basis states
$$\mathcal{L}[S] = \frac{1}{d} \sum_{i=1}^d (1-\Vert U(t)\phi_i\Vert^2)=1 - \frac{1}{d}\mathrm{tr}(S^\dagger S).$$

If we work with positive maps, we can do something similar. We assume that we have a representation of the projected positive map, which only contains the components in the target basis
$${\cal E}_{ij,kl} = \langle{i}|{\cal E}(|k\rangle\langle{l}|)|j\rangle, $$
and estimate the leakage
$${\cal L}[{\cal E}] = 1-\frac{1}{d}\sum_{i,k} {\cal E}_{ii,kk}$$

In [ ]:
# file: seeq/fidelity.py

def leakage(S):
    """Compute the leakage outside the computational space, for a matrix
    S that connects input and output states in the computational basis,
    and which is in general not unitary."""
    if S.ndim == 2:
        d = S.shape[0]
        return np.abs(1 - np.vdot(S, S)/d)
    elif S.ndim == 4:
        d = S.shape[0]
        return np.abs(1 - np.einsum('iijj', S)/d)
    else:
        raise ValueError('Not a valid unitary or positive map')

## Fidelity without phases

Sometimes we want to study transformations without phases that can be corrected. Other times, we want to separate local phases from nonlocal ones. The first routine extracts a diagonal operation that only contains phases relative to the first state.

In [ ]:
# file: seeq/fidelity.py

def extract_phases(T):
    """Extract diagonal phases from a unitary operation or superoperator"""
    if T.ndim == 2:
        # Unitary operator
        v = np.diag(T) / T[0,0]
        v /= np.abs(v)
        return np.diag(v)
    elif T.ndim == 4:
        # Superoperator
        return extract_phases(T[:,0,:,0])
    raise ValueError('Not a valid unitary or positive map.')

With this, we can take a transformation and remove those phases.

In [ ]:
# file: seeq/fidelity.py

def remove_phases(T):
    """Eliminate diagonal phases from a unitary operation or superoperator"""
    inv = extract_phases(T).conj()
    if T.ndim == 4:
        d = inv.shape[0]
        return np.einsum('ij,kl,jlmn', inv, inv.conj(), T)
    return inv @ T

This allows us to construct an average fidelity where those phases are removed. This average fidelity is very useful to study adiabatic processes where the dynamical phases are not relevant, but we want to understand how basis states are mapped to each other.

In [ ]:
# file: seeq/fidelity.py

def avg_fidelity_no_phases(T):
    return avg_fidelity(remove_phases(T))